# Lógica proposicional clásica del profesor Pantano

### Inicio de las pruebas con inteligencia artificial

#### Carga de variables de entorno

In [24]:
import os
from dotenv import load_dotenv
from langchain_community.llms import Cohere

load_dotenv()

os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")

#### Instanciación del chat de IA

In [25]:
from langchain_cohere.chat_models import ChatCohere

llm = ChatCohere(model="command-r-plus", language="es", max_tokens=100)


#### Prompt

In [26]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente en el estudio de la materia: {tema}. "+
            "'Proposición' es una expresión que afirma algo, verdadero o falso, pero no ambas cosas a la vez. "+
            "Por ejemplo: '6 es un número primo' es una proposición, pero 'x=2' no. "+
            "Responde con la mayor síntesis posible "+
            "a la pregunta de si el input es o no una proposición."
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | llm

#### Funciones auxiliares

In [27]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

#### Input y resultado

In [28]:
input = "Jorge Luis Borges escribió el libro de cuentos 'Ficciones'"

result = with_message_history.invoke(
    {"tema": "Lógica Proposicional Clásica: ", 
     "input": input },
    config={"configurable": {"session_id": "abc123"}},
)

print(result.content)

Sí.


In [29]:
input2 = "¿Cómo?"
result2 = with_message_history.invoke(
    {"tema": "Lógica Proposicional Clásica: ", 
     "input": input2 },
    config={"configurable": {"session_id": "abc123"}},
)
print(result2.content)

Sí, es una proposición.


### Términos categoremáticos (no lógicos) y sincategoremáticos (lógicos)

In [39]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente en el estudio de la materia: {tema}. "+
            "Los términos no lógicos, también llamados categoremáticos, son aquellos que tienen "+
            "significado por sí mismos o que nombran objetos reales o imaginarios, como por ejemplo "+
            "'simpático' o 'árbitro de fútbol'. "+
            "Los términos lógicos, también llamados sincategoremáticos, no tienen significado por sí "+
            "mismos y sólo lo adquieren acompañando, uniendo, estructurado, a los términos no lógicos. Así, por "+
            "ejemplo, 'ningún', 'todos', 'es', 'no', etc. son términos lógicos. "+
            "Responde con la mayor síntesis posible "+
            "qué elementos del input son categoremáticos y cuáles sincategoremáticos."
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

runnable = prompt | llm

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [40]:
input = "Ningún perro es gato"
result = with_message_history.invoke(
    {"tema": "Lógica Proposicional Clásica: ", 
     "input": input },
    config={"configurable": {"session_id": "abc123"}},
)
print(result.content)

'Perro' y 'gato' son términos categoremáticos. 'Ningún' y 'es' son términos sincategoremáticos.


#### Clasificación de las proposiciones

In [43]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Las proposiciones, a su vez, se pueden clasificar de distintas maneras. Un grupo de "+
            "proposiciones particularmente importante, denominadas proposiciones categóricas, son las "+
            "siguientes: "+
            "Universal afirmativa: Todo S es P (de tipo A) "+
            "Universal negativa: Ningún S es P (de tipo E) "+
            "Particular afirmativa: Algún S es P (de tipo I) "+
            "Particular negativa: Algún S es P (de tipo O) "+
            "Responde con la mayor síntesis posible "+
            "qué tipo de proposición es la provista en el input."
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | llm

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [44]:
input = "No todo lo que brilla es oro"
result = with_message_history.invoke(
    {"tema": "Lógica Proposicional Clásica: ", 
     "input": input },
    config={"configurable": {"session_id": "abc123"}},
)
print(result.content)

Particular negativa (tipo O)


#### Inferencias por oposición

##### Contrarias

In [47]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Contrarias: dos proposiciones contrarias pueden ser simultáneamente falsas, pero no "+
            "simultaneamente verdaderas. Si una es verdadera, la otra es falsa. Por ejemplo, si es verdadero que 'Todos los peces "+
            "viven en el agua' es falsa la proposición 'Ningún pez vive en el agua', pero si una de las contrarias es "+
            "falsa, nada se puede establecer acerca del valor de la otra. Habrá casos en que una es falsa y la otra "+
            "también, y casos en que una será falsa y la otra verdadera."+
            "Responde con fundamento "+
            "dada la expresión del input, ¿se puede determinar el valor de verdad de su expresión contraria?"
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | llm

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [48]:
input = "Todos los peces viven en el agua"
result = with_message_history.invoke(
    {"tema": "Lógica Proposicional Clásica: ", 
     "input": input },
    config={"configurable": {"session_id": "abc123"}},
)
print(result.content)

La expresión "Todos los peces viven en el agua" es una proposición universal afirmativa, que afirma que todos los peces, sin excepción, viven en el agua. Su expresión contraria sería una proposición universal negativa, que negaría que todos los peces viven en el agua.

La expresión contraria de "Todos los peces viven en el agua" sería "Ningún pez vive en el agua". En este caso, sí se puede determinar el valor de verdad de su expresión contraria. Si la proposición original es verdadera, entonces su contraria es necesariamente falsa. Por lo tanto, si es verdadero que "Todos los peces viven en el agua", entonces es falso que "Ningún pez vive en el agua".

En resumen, dada la expresión "Todos los peces viven en el agua", se puede determinar que su expresión contraria ("Ningún pez vive en el agua") es falsa si la proposición original es verdadera.


#### Subcontrarias

In [63]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Subcontrarias: dos proposiciones subcontrarias pueden ser simultáneamente verdaderas, "+
            "pero no falsas. Si una es falsa, la otra es verdadera necesariamente. "+
            "Responde con fundamento "+
            "dada la expresión del input, ¿se puede determinar el valor de verdad de su expresión subcontraria?"
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | llm

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [64]:
input = "Algunos piratas visten saco y corbata"
result = with_message_history.invoke(
    {"tema": "Lógica Proposicional Clásica: ", 
     "input": input },
    config={"configurable": {"session_id": "abc123"}},
)
print(result.content)

Sí, se puede determinar el valor de verdad de la expresión subcontraria. 

La expresión original, "Algunos piratas visten saco y corbata", indica que existe al menos un pirata que viste de esa manera. La expresión subcontraria sería "Algunos piratas no visten saco y corbata", y su valor de verdad puede determinarse a partir de la información dada.

Si la expresión original es verdadera, entonces es razonable asumir que existen piratas que no visten saco y corbata. Por lo tanto, la expresión subcontraria "Algunos piratas no visten saco y corbata" también sería verdadera.

Por otro lado, si la expresión original fuera falsa, y ningún pirata vistiera saco y corbata, entonces necesariamente todos los piratas no visten de esa manera. En este caso, la expresión subcontraria "Algunos piratas no visten saco y corbata" seguiría siendo verdadera.

En resumen, la expresión subcontraria "Algunos piratas no visten saco y corbata" sería verdadera tanto si la expresión original es verdadera como si e

##### Contradictorias

In [55]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Contradictorias: no pueden ser ni simultáneamente "+
            "verdaderas ni simultáneamente falsas. Si una es falsa, la otra es necesariamente verdadera, "+
            "y si una es verdadera, la otra es falsa necesariamente. "+
            "Por ejemplo, de la falsedad de 'Todos los astronautas son "+
            "norteamericanos” se infiere la verdad de 'Algunos astronautas no son norteamericanos'"+
            "Dada la expresión del input, ¿se puede determinar el valor de verdad de su expresión contradictoria?"
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | llm

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [56]:
input = "Todos los domingos los Benvenutti almuerzan pasta"
result = with_message_history.invoke(
    {"tema": "Lógica Proposicional Clásica: ", 
     "input": input },
    config={"configurable": {"session_id": "abc123"}},
)
print(result.content)

La expresión contradictoria de la oración "Todos los domingos los Benvenutti almuerzan pasta" sería:

"Algunos domingos los Benvenutti no almuerzan pasta".

Esta expresión contradictoria es verdadera si la original es falsa, y falsa si la original es verdadera. Por ejemplo, si los Benvenutti a veces comen algo diferente a la pasta los domingos, entonces la primera oración es falsa y la expresión contradictoria es verdadera. Por otro lado, si los Benvenutti siempre comen pasta los domingos sin excepción, entonces la primera oración es verdadera y la expresión contradictoria es falsa.


##### Subalternas

In [61]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Subalternas: se llama subalternante a la proposición universal y su alternada a la particular "
            "correspondiente. Si la subalternante es verdadera, la subalternada es verdadera, y si la subalternante "+
            "es falsa, nada se puede inferir para la subalternada. Si la subalternada es falsa, la subalternante es "+
            "falsa, y si la subalternada es verdadera, nada se puede inferir que la subalternante."+
            "Dada la expresión del input, ¿se puede determinar el valor de verdad de su expresión subalterna?"
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
runnable = prompt | llm

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [62]:
input = "Ninguna persona nace siendo mala"
result = with_message_history.invoke(
    {"tema": "Lógica Proposicional Clásica: ", 
     "input": input },
    config={"configurable": {"session_id": "abc123"}},
)
print(result.content)

Verdadera. Si la proposición universal "Ninguna persona nace siendo mala" es verdadera, entonces su proposición particular correspondiente, "Algunas personas no nacen siendo malas", también es verdadera.
